In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
#from psycopg2 import extras as ex
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from sklearn.preprocessing import *
import datetime

## prepare data

In [2]:
# connect to the postgresql
db_connection = psycopg2.connect(host='127.0.0.1',dbname="chicago_business_intelligence", user="postgres" , password="12345")
cursor = db_connection.cursor()

In [3]:
# get data
select_data = (
    """
        SELECT * FROM "neighborhood_community_zip";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_1 = pd.DataFrame(result_1)
df_1.columns = ["id","geo_id","zipcode","community","neighborhood"]
df_1 = df_1[["zipcode","community","neighborhood"]]
df_1.zipcode = df_1.zipcode.astype("int")
df_1

,zipcode,community,neighborhood
0,60601,Loop,Printer's Row
1,60602,Loop,Printer's Row
2,60603,Loop,Printer's Row
3,60604,Loop,Printer's Row
4,60605,Loop,Printer's Row
...,...,...,...
306,60652,Ashburn,Parkview
307,60652,Ashburn,Scottsdale
308,60652,Ashburn,Wrightwood
309,60656,O'Hare,Schorsch Forest View


In [4]:
# get data
select_data = (
    """
        SELECT * FROM "taxi_trip_total";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_2 = pd.DataFrame(result_1)
df_2.columns = ["id","trip_id","trip_start_timestamp","trip_end_timestamp","pickup_centroid_latitude","pickup_centroid_longitude",
                   "dropoff_centroid_latitude","dropoff_centroid_longitude","pickup_zip_code","dropoff_zip_code"]
df_2 = df_2[["trip_start_timestamp","trip_end_timestamp","pickup_zip_code","dropoff_zip_code"]]
df_2.pickup_zip_code = df_2.pickup_zip_code.astype("int")
df_2.dropoff_zip_code = df_2.dropoff_zip_code.astype("int")
df_2

,trip_start_timestamp,trip_end_timestamp,pickup_zip_code,dropoff_zip_code
0,2022-03-01 00:00:00+08:00,2022-03-01 00:00:00+08:00,60666,60666
1,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60610,60657
2,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60657,60616
3,2022-03-01 00:00:00+08:00,2022-03-01 01:00:00+08:00,60610,60610
4,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60666,60604
...,...,...,...,...
3239,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60607,60603
3240,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60643,60616
3241,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60622,60610
3242,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60610,60604


## process data

In [5]:
#zipcode
#divide into pickup and dropoff
df_2_pick = df_2[["trip_start_timestamp","trip_end_timestamp","pickup_zip_code"]]
df_2_pick.columns = ["trip_start_timestamp","trip_end_timestamp","zipcode"]
df_2_off = df_2[["trip_start_timestamp","trip_end_timestamp","dropoff_zip_code"]]
df_2_off.columns = ["trip_start_timestamp","trip_end_timestamp","zipcode"]
#merge
df_2_total = df_2_pick.append(df_2_pick)
df_2_total = df_2_total.reset_index(drop=True)
df_2_total["ymd"] = df_2_total["trip_start_timestamp"].apply(lambda x:x.strftime("%Y-%m-%d"))
df_2_total = df_2_total[["ymd","zipcode"]]
df_2_total["help"] = 0
df_2_total_zipcode = df_2_total
df_2_total_zipcode

,ymd,zipcode,help
0,2022-03-01,60666,0
1,2022-03-01,60610,0
2,2022-03-01,60657,0
3,2022-03-01,60610,0
4,2022-03-01,60666,0
...,...,...,...
6483,2022-02-28,60607,0
6484,2022-02-28,60643,0
6485,2022-02-28,60622,0
6486,2022-02-28,60610,0


In [6]:
#neighborhood
#divide into pickup and dropoff
df_2_pick = df_2[["trip_start_timestamp","trip_end_timestamp","pickup_zip_code"]]
df_2_pick.columns = ["trip_start_timestamp","trip_end_timestamp","zipcode"]
df_2_off = df_2[["trip_start_timestamp","trip_end_timestamp","dropoff_zip_code"]]
df_2_off.columns = ["trip_start_timestamp","trip_end_timestamp","zipcode"]
#merge
df_2_total = df_2_pick.append(df_2_pick)
df_2_total = df_2_total.reset_index(drop=True)
df_2_total = df_2_total.merge(df_1,how='inner', on='zipcode')
df_2_total["ymd"] = df_2_total["trip_start_timestamp"].apply(lambda x:x.strftime("%Y-%m-%d"))
df_2_total = df_2_total[["ymd","community"]]
df_2_total["help"] = 0
df_2_total_community = df_2_total
df_2_total_community

,ymd,community,help
0,2022-03-01,O'Hare,0
1,2022-03-01,O'Hare,0
2,2022-03-01,O'Hare,0
3,2022-03-01,O'Hare,0
4,2022-03-01,O'Hare,0
...,...,...,...
37233,2022-02-28,Beverly,0
37234,2022-02-28,Morgan Park,0
37235,2022-02-28,Morgan Park,0
37236,2022-02-28,Morgan Park,0


## zip_report

In [7]:
# record
rp_df_2_total = df_2_total_zipcode[['ymd',"zipcode","help"
                        ]].groupby(by=["ymd","zipcode"],dropna=True)["help"].count().reset_index()
rp_df_2_total.columns=['ymd',"zipcode","trip_count"]
rp_df_2_total

,ymd,zipcode,trip_count
0,2022-02-28,60018,2
1,2022-02-28,60601,84
2,2022-02-28,60603,174
3,2022-02-28,60604,406
4,2022-02-28,60605,64
...,...,...,...
58,2022-03-01,60622,2
59,2022-03-01,60638,4
60,2022-03-01,60657,2
61,2022-03-01,60660,2


In [8]:
# predict
avg_df = rp_df_2_total.groupby(by=["zipcode"],dropna=True)["trip_count"].mean().reset_index()
avg_df["predict_date"] = df_2["trip_start_timestamp"][0]+ datetime.timedelta(days=1)
avg_df["predict_date"] = avg_df["predict_date"].apply(lambda x:x.strftime("%Y-%m-%d"))
avg_df.columns = ["zipcode","predict_count","predict_date"]
avg_df

,zipcode,predict_count,predict_date
0,60018,2.0,2022-03-02
1,60601,84.0,2022-03-02
2,60603,174.0,2022-03-02
3,60604,204.0,2022-03-02
4,60605,64.0,2022-03-02
5,60607,208.0,2022-03-02
6,60608,70.0,2022-03-02
7,60609,68.0,2022-03-02
8,60610,397.0,2022-03-02
9,60611,132.0,2022-03-02


## community_report

In [9]:
# record
rp_df_2_total = df_2_total_community[['ymd',"community","help"
                        ]].groupby(by=["ymd","community"],dropna=True)["help"].count().reset_index()
rp_df_2_total.columns=['ymd',"community","trip_count"]
rp_df_2_total

,ymd,community,trip_count
0,2022-02-28,Albany Park,360
1,2022-02-28,Armour Square,476
2,2022-02-28,Ashburn,120
3,2022-02-28,Austin,328
4,2022-02-28,Avalon Park,300
...,...,...,...
58,2022-03-01,Near South Side,4
59,2022-03-01,Near West Side,60
60,2022-03-01,North Center,4
61,2022-03-01,O'Hare,12


In [10]:
# predict
avg_df = rp_df_2_total.groupby(by=["community"],dropna=True)["trip_count"].mean().reset_index()
avg_df["predict_date"] = df_2["trip_start_timestamp"][0]+ datetime.timedelta(days=1)
avg_df["predict_date"] = avg_df["predict_date"].apply(lambda x:x.strftime("%Y-%m-%d"))
avg_df.columns = ["community","predict_count","predict_date"]
avg_df

,community,predict_count,predict_date
0,Albany Park,360.0,2022-03-02
1,Armour Square,476.0,2022-03-02
2,Ashburn,120.0,2022-03-02
3,Austin,328.0,2022-03-02
4,Avalon Park,300.0,2022-03-02
5,Beverly,62.0,2022-03-02
6,Calumet Heights,150.0,2022-03-02
7,Chatham,112.0,2022-03-02
8,Chicago Lawn,120.0,2022-03-02
9,Clearing,154.0,2022-03-02
